In [2]:
pip install xlsxwriter

In [1]:
pip install sqlalchemy pymysql pandas openpyxl


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/2.1 MB 6.9 MB/s eta 0:00:01
   --------------- ------------------------ 0.8/2.1 MB 10.1 MB/s eta 0:00:01
   -------------------------- ------------- 1.4/2.1 MB 10.7 MB/s eta 0:00:01
   ------------------------------------- -- 1.9/2.1 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 11.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ---------------------------------------- 45.0/45.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/293.6 kB ? eta -:--:--
   ---------------------------------------- 293.6/293.6 kB 8.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# 파일 경로 설정

# 엑셀 파일 읽기
df = pd.read_excel(file_path, sheet_name='CS Receiving TAT')

# Quantity 컬럼 내용 확인
quantity_values = df['Quantity'].head(20)
print('Initial Quantity values:', quantity_values)

# Quantity 컬럼을 정수로 변환 (변환되지 않는 값은 NaN으로 설정)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

# 변환되지 않은 값들 확인
invalid_values = df[df['Quantity'].isna()]['Quantity']
print('Invalid Quantity values:', invalid_values)

# NaN 값을 0으로 대체
df['Quantity'] = df['Quantity'].fillna(0).astype(int)

# 변환 후 데이터 확인
print('Converted Quantity values:', df['Quantity'].head(20))


Initial Quantity values: 0     1
1     1
2     1
3     1
4     1
5     3
6     1
7     4
8     1
9     1
10    2
11    1
12    1
13    3
14    1
15    2
16    1
17    1
18    1
19    2
Name: Quantity, dtype: int64
Invalid Quantity values: Series([], Name: Quantity, dtype: int64)
Converted Quantity values: 0     1
1     1
2     1
3     1
4     1
5     3
6     1
7     4
8     1
9     1
10    2
11    1
12    1
13    3
14    1
15    2
16    1
17    1
18    1
19    2
Name: Quantity, dtype: int32
